In [1]:
!pip install gensim
!pip install python-Levenshtein

     |████████████████████████████████| 24.1 MB 3.1 MB/s eta 0:00:01    |███                             | 2.3 MB 2.3 MB/s eta 0:00:10     |█████████▍                      | 7.0 MB 2.7 MB/s eta 0:00:07     |█████████████████████▏          | 15.9 MB 3.7 MB/s eta 0:00:03
     |████████████████████████████████| 58 kB 218 kB/s eta 0:00:011
     |████████████████████████████████| 50 kB 719 kB/s  eta 0:00:01
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=183038 sha256=fc9ae914d2de3687886397f95a41f457e4cbaf3e80008fe07404ef568fc5ad8c
  Stored in directory: /home/aj/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [2]:
import gensim
import pandas as pd

In [4]:
df = pd.read_json("Cell_Phones_and_Accessories_5.json", lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [5]:
df.shape

(194439, 9)

In [6]:
#Simple Preprocessing & Tokenization
review_text = df.reviewText.apply(gensim.utils.simple_preprocess)


In [7]:
review_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [8]:
review_text.loc[0]

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [9]:
df.reviewText.loc[0]


"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [11]:
#Training the Word2Vec Model
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,
)

In [12]:
#Build Vocabulary

model.build_vocab(review_text, progress_per=1000)

In [13]:
#Train the Word2Vec Model
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)


(61505383, 83868975)

In [14]:
#Save the Model
model.save("./word2vec-amazon-cell-accessories-reviews-short.model")


In [15]:
#Finding Similar Words and Similarity between words

model.wv.most_similar("bad")


[('terrible', 0.6806171536445618),
 ('horrible', 0.5982799530029297),
 ('shabby', 0.5886836647987366),
 ('good', 0.5830838680267334),
 ('legit', 0.5284755825996399),
 ('poor', 0.5239996314048767),
 ('disappointing', 0.5200459361076355),
 ('sad', 0.5158399343490601),
 ('lame', 0.5141434669494629),
 ('ok', 0.5137726664543152)]

In [16]:
model.wv.similarity(w1="cheap", w2="inexpensive")


0.53287905

In [17]:
model.wv.similarity(w1="great", w2="good")


0.7883202